# Quadrature Phase Modulation

In [ ]:
from disiple.signals import TimeSignal, Spectrum
import numpy as np
from bokeh.plotting import show
from bokeh.layouts import row, column, gridplot
from bokeh.models import CustomJS, Div, Slider, TextInput

In [ ]:
samples_per_bit = 1000
sample_rate = 22050 # [samples per second=Hz]
bit_rate = sample_rate / samples_per_bit

### Message Signal

In [ ]:
bit_stream = [0, 0, 0, 1, 1, 0, 1, 1, 0]
samples_mod = np.repeat(bit_stream, samples_per_bit)
mod_signal = TimeSignal(samples_mod, sample_rate)
mod_spec = Spectrum.from_timesignal(mod_signal, dB=False)

### Split Message Signal in Two

In [ ]:
if len(bit_stream) % 2 == 1:
    bit_stream.append(0)
in_phase_stream = bit_stream[1::2]
samples_in_phase = np.repeat(in_phase_stream, samples_per_bit)
in_phase_signal = TimeSignal(samples_in_phase, sample_rate)
in_phase_spec = Spectrum.from_timesignal(in_phase_signal, dB=False)

quad_phase_stream = bit_stream[0::2]
samples_quad_phase = np.repeat(quad_phase_stream, samples_per_bit)
quad_phase_signal = TimeSignal(samples_quad_phase, sample_rate)
quad_phase_spec = Spectrum.from_timesignal(quad_phase_signal, dB=False)

### Two Carrier Signals

In [ ]:
amp_carr = 1
freq_carr = 150
qpsk_times = np.arange(0, len(samples_in_phase)) / sample_rate

samples_in_carr = amp_carr * np.sin(2*np.pi*freq_carr*qpsk_times)
in_carr_signal = TimeSignal(samples_in_carr, sample_rate)
in_carr_spec = Spectrum.from_timesignal(in_carr_signal, dB=False)

samples_quad_carr = amp_carr * np.cos(2*np.pi*freq_carr*qpsk_times)
quad_carr_signal = TimeSignal(samples_quad_carr, sample_rate)
quad_carr_spec = Spectrum.from_timesignal(quad_carr_signal, dB=False)

### Quadrature Phase Shift Keying

In [ ]:
samples_in_bpsk = np.where(samples_in_phase == 1, samples_in_carr, -samples_in_carr)
in_bpsk_signal = TimeSignal(samples_in_bpsk, sample_rate)
in_bpsk_spec = Spectrum.from_timesignal(in_bpsk_signal, dB=False)

samples_quad_bpsk = np.where(samples_quad_phase == 1, samples_quad_carr, -samples_quad_carr)
quad_bpsk_signal = TimeSignal(samples_quad_bpsk, sample_rate)
quad_bpsk_spec = Spectrum.from_timesignal(quad_bpsk_signal, dB=False)

qpsk_signal = in_bpsk_signal + quad_bpsk_signal
qpsk_spec = Spectrum.from_timesignal(qpsk_signal, dB=False)

### Create Figures

In [ ]:
amp_range = (-0.04, 1.04)
mag_range = (0, amp_carr/2 * 1.05)
freq_range = (0, 300)

mod_fig = mod_signal.plot(title='Message Signal', y_range=amp_range, active_inspect=None)
mod_spec_fig = mod_spec.plot(title='Magnitude Spectrum of Message Signal', x_range=freq_range, y_range=mag_range, active_inspect=None)

in_phase_fig = in_phase_signal.plot(title='In-Phase Message Signal', y_range=amp_range, active_inspect=None, line_color='darkseagreen')
in_phase_spec_fig = in_phase_spec.plot(title='Magnitude Spectrum of In-Phase Message Signal', x_range=freq_range, y_range=mag_range, active_inspect=None, line_color='darkseagreen')
quad_phase_fig = quad_phase_signal.plot(title='Quadrature-Phase Message Signal', y_range=amp_range, active_inspect=None, line_color='indianred')
quad_phase_spec_fig = quad_phase_spec.plot(title='Magnitude Spectrum of Quadrature-Phase Message Signal', x_range=freq_range, y_range=mag_range, active_inspect=None, line_color='indianred')

in_carr_fig = in_carr_signal.plot(title='In-Phase Carrier Signal', active_inspect=None, line_color='darkolivegreen')
in_carr_spec_fig = in_carr_spec.plot(title='Magnitude Spectrum of In-Phase Carrier Signal', x_range=freq_range, y_range=mag_range, active_inspect=None, line_color='darkolivegreen')
quad_carr_fig = quad_carr_signal.plot(title='Quadrature-Phase Carrier Signal', active_inspect=None, line_color='crimson')
quad_carr_spec_fig = quad_carr_spec.plot(title='Magnitude Spectrum of Quadrature-Phase Carrier Signal', x_range=freq_range, y_range=mag_range, active_inspect=None, line_color='crimson')

in_bpsk_fig = in_bpsk_signal.plot(title='In-Phase BPSK Signal', active_inspect=None, line_color='darkgreen')
in_bpsk_spec_fig = in_bpsk_spec.plot(title='Magnitude Spectrum of In-Phase BPSK Signal', x_range=freq_range, y_range=mag_range, active_inspect=None, line_color='darkgreen')
quad_bpsk_fig = quad_bpsk_signal.plot(title='Quadrature-Phase BPSK Signal', active_inspect=None, line_color='darkred')
quad_bpsk_spec_fig = quad_bpsk_spec.plot(title='Magnitude Spectrum of Quadrature-Phase BPSK Signal', x_range=freq_range, y_range=mag_range, active_inspect=None, line_color='darkred')

qpsk_fig = qpsk_signal.plot(title='QPSK Signal', active_inspect=None, line_color='darkgoldenrod')
qpsk_spec_fig = qpsk_spec.plot(title='Magnitude Spectrum of QPSK Signal', x_range=freq_range, y_range=mag_range, active_inspect=None, line_color='darkgoldenrod')

### Link Frequency Axes

In [ ]:
from itertools import product
def link_x_axes(figs):
    for fig1, fig2 in product(figs, figs):
        fig1.x_range.js_link('start', fig2.x_range, 'start')
        fig1.x_range.js_link('end', fig2.x_range, 'end')
link_x_axes({in_phase_spec_fig, quad_phase_spec_fig, in_carr_spec_fig, quad_carr_spec_fig, in_bpsk_spec_fig, quad_bpsk_spec_fig, qpsk_spec_fig})
link_x_axes({in_phase_fig, quad_phase_fig, in_carr_fig, quad_carr_fig, in_bpsk_fig, quad_bpsk_fig, qpsk_fig})

### Display Figures

In [ ]:
width = 700
plot = column(
    row(mod_fig, mod_spec_fig, width=width),
    row(
        gridplot([in_phase_fig, quad_phase_fig, in_carr_fig, quad_carr_fig, in_bpsk_fig, quad_bpsk_fig], ncols=2, width=width//2), 
        gridplot([in_phase_spec_fig, quad_phase_spec_fig, in_carr_spec_fig, quad_carr_spec_fig, in_bpsk_spec_fig, quad_bpsk_spec_fig], ncols=2, width=width//2),
    ),
    row(qpsk_fig, qpsk_spec_fig, width=width),
)
show(plot)

### Add Interaction

In [ ]:
bit_input = TextInput(value=','.join([str(bit) for bit in bit_stream]), title='Message:')
samples_per_bit_slider = Slider(start=500, end=1500, value=samples_per_bit, step=1, title='Samples per bit')
freq_carr_slider = Slider(start=50, end=250, value=freq_carr, step=1, title='Carrier Frequency')
bit_rate_div = Div(text=f'Bit rate: <b>{bit_rate:.2f} bps</b>', width=80, height=10)

callback = CustomJS(args=dict(
    modSource=mod_fig.renderers[0].data_source,
    modSpecSource=mod_spec_fig.renderers[0].data_source,
    inPhaseSource=in_phase_fig.renderers[0].data_source,
    inPhaseSpecSource=in_phase_spec_fig.renderers[0].data_source,
    quadPhaseSource=quad_phase_fig.renderers[0].data_source,
    quadPhaseSpecSource=quad_phase_spec_fig.renderers[0].data_source,
    inCarrSource=in_carr_fig.renderers[0].data_source,
    inCarrSpecSource=in_carr_spec_fig.renderers[0].data_source,
    quadCarrSource=quad_carr_fig.renderers[0].data_source,
    quadCarrSpecSource=quad_carr_spec_fig.renderers[0].data_source,
    inBpskSource=in_bpsk_fig.renderers[0].data_source,
    inBpskSpecSource=in_bpsk_spec_fig.renderers[0].data_source,
    quadBpskSource=quad_bpsk_fig.renderers[0].data_source,
    quadBpskSpecSource=quad_bpsk_spec_fig.renderers[0].data_source,
    qpskSource=qpsk_fig.renderers[0].data_source,
    qpskSpecSource=qpsk_spec_fig.renderers[0].data_source,
    ampCarr=amp_carr,
    sampleRate=sample_rate,
    bitInput=bit_input,
    samplesPerBitSlider=samples_per_bit_slider,
    freqCarrSlider=freq_carr_slider,
    bitRateDiv=bit_rate_div,
    ), code="""
    const samplesPerBit = samplesPerBitSlider.value;
    const bitRate = sampleRate / samplesPerBit;
    bitRateDiv.text = `Bit rate: <b>${bitRate.toFixed(2)} bps</b>`;
    
    let bitStream = bitInput.value.split(",").map(Number);
    const bitSamples = [].concat(...bitStream.map((el) => Array(samplesPerBit).fill(el)));
    if (bitStream.length % 2 === 1) {
        bitStream.push(0);
    }
    const inPhaseStream = bitStream.filter((el, idx) => idx % 2 === 1);
    const inPhaseSamples = [].concat(...inPhaseStream.map((el) => Array(samplesPerBit).fill(el)));
    const quadPhaseStream = bitStream.filter((el, idx) => idx % 2 === 0);
    const quadPhaseSamples = [].concat(...quadPhaseStream.map((el) => Array(samplesPerBit).fill(el)));

    const bitTimeStamps = bitSamples.map((_, idx) => idx/sampleRate);
    const qpskTimeStamps = inPhaseSamples.map((_, idx) => idx/sampleRate);
    const freqCarr = freqCarrSlider.value;
    const inCarrSamples = sineWave(qpskTimeStamps, ampCarr, freqCarr);
    const quadCarrSamples = cosineWave(qpskTimeStamps, ampCarr, freqCarr);

    const inBpskSamples = inPhaseSamples.map((bit, idx) => bit != 0 ? inCarrSamples[idx] : -inCarrSamples[idx]);
    const quadBpskSamples = quadPhaseSamples.map((bit, idx) => bit != 0 ? quadCarrSamples[idx] : -quadCarrSamples[idx]);
    const qpskSamples = inBpskSamples.map((el, idx) => el + quadBpskSamples[idx]);
    
    modSource.data = {'x': bitTimeStamps, 'y': bitSamples};
    inPhaseSource.data = {'x': qpskTimeStamps, 'y': inPhaseSamples};
    quadPhaseSource.data = {'x': qpskTimeStamps, 'y': quadPhaseSamples};
    inCarrSource.data = {'x': qpskTimeStamps, 'y': inCarrSamples};
    quadCarrSource.data = {'x': qpskTimeStamps, 'y': quadCarrSamples};
    inBpskSource.data = {'x': qpskTimeStamps, 'y': inBpskSamples};
    quadBpskSource.data = {'x': qpskTimeStamps, 'y': quadBpskSamples};
    qpskSource.data = {'x': qpskTimeStamps, 'y': qpskSamples};

    const fourier = new Fourier(nextpow2(bitSamples.length));
    const freqs = fourier.rfftFreqs(sampleRate);
    modSpecSource.data = {'bin_unit': freqs, 'frequency': freqs, 'magnitude': fourier.magnitudeSpectrum(bitSamples)};
    
    const qpskFourier = new Fourier(nextpow2(qpskSamples.length));
    const qpskFreqs = qpskFourier.rfftFreqs(sampleRate);
    inPhaseSpecSource.data = {'bin_unit': qpskFreqs, 'frequency': qpskFreqs, 'magnitude': qpskFourier.magnitudeSpectrum(inPhaseSamples)};
    quadPhaseSpecSource.data = {'bin_unit': qpskFreqs, 'frequency': qpskFreqs, 'magnitude': qpskFourier.magnitudeSpectrum(quadPhaseSamples)};
    inCarrSpecSource.data = {'bin_unit': qpskFreqs, 'frequency': qpskFreqs, 'magnitude': qpskFourier.magnitudeSpectrum(inCarrSamples)};
    quadCarrSpecSource.data = {'bin_unit': qpskFreqs, 'frequency': qpskFreqs, 'magnitude': qpskFourier.magnitudeSpectrum(quadCarrSamples)};
    inBpskSpecSource.data = {'bin_unit': qpskFreqs, 'frequency': qpskFreqs, 'magnitude': qpskFourier.magnitudeSpectrum(inBpskSamples)};
    quadBpskSpecSource.data = {'bin_unit': qpskFreqs, 'frequency': qpskFreqs, 'magnitude': qpskFourier.magnitudeSpectrum(quadBpskSamples)};
    qpskSpecSource.data = {'bin_unit': qpskFreqs, 'frequency': qpskFreqs, 'magnitude': qpskFourier.magnitudeSpectrum(qpskSamples)};
""")
bit_input.js_on_change('value', callback)
samples_per_bit_slider.js_on_change('value', callback)
freq_carr_slider.js_on_change('value', callback)

width = 700
interactive_plot = column(
    row(bit_input, samples_per_bit_slider, freq_carr_slider, bit_rate_div, width=width),
    row(mod_fig, mod_spec_fig, width=width),
    row(
        gridplot([in_phase_fig, quad_phase_fig, in_carr_fig, quad_carr_fig, in_bpsk_fig, quad_bpsk_fig], ncols=2, width=width//2), 
        gridplot([in_phase_spec_fig, quad_phase_spec_fig, in_carr_spec_fig, quad_carr_spec_fig, in_bpsk_spec_fig, quad_bpsk_spec_fig], ncols=2, width=width//2),
    ),
    row(qpsk_fig, qpsk_spec_fig, width=width),
)
show(interactive_plot)

In [ ]:
from bokeh.plotting import save
from bokeh.resources import INLINE
template = f"""
{{% block postamble %}}
    <script src="fourier.js"></script>
    <script src="modulation-schemes.js"></script>
{{% endblock %}} """
save(interactive_plot, filename='qpsk.html', title='Quadrature Phase Shift Keying', resources=INLINE, template=template)